In [5]:
import Axios from 'axios';
import * as fs from 'fs/promises';

const loadPage = async (title: string) => {
  const SEARCH_URL = "https://www.amazon.com/s";
  const result = await Axios.get(SEARCH_URL, {
    params: {
      i: 'digital-text',
      k: title,
    },
    validateStatus: () => true,
    headers: {'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-gpc': '1',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9'}
  });

  await fs.writeFile('testContent.html', result.data);
  return result.data;
};

In [11]:
import * as cheerio from 'cheerio';


interface AmazonSearchResult {
  asin: string;
  title: string;
  imageUrl: string;
  author?: string;
  published?: string;
  productPath: string;
}

function parseAmazonSearchResults(content: string): AmazonSearchResult[] {
  const results = [] as AmazonSearchResult[];
  const $ = cheerio.load(content);

  const resultContainer = $('div.s-search-results.s-main-slot').first();
  
  // Look for no results banner
  const banner = resultContainer.children('div.a-section').first().text();
  if (banner.includes('No results for ') || banner.includes('Showing results from All Departments')) {
    return results;
  }

  resultContainer.children('div[data-component-type=s-search-result]').each((i, elem) => {
    const e = $(elem);
    const asin = e.attr('data-asin');

    const imageRecord = e.find('img[class=s-image]').first();
    const title = imageRecord.attr('alt');
    const imageUrl = imageRecord.attr('src');

    const imageContainer = e.find('span[data-component-type=s-product-image]').first();
    const productLink = imageContainer.find('a').first();
    let productPath = productLink.attr('href');
    productPath = productPath?.substring(0, productPath.lastIndexOf('/'))

    const headerElement = e.find('.s-title-instructions-style').first();
    const metaElements = $(headerElement[0].children[1]).find('.a-row').first();
    const author = $(metaElements).find('.s-link-style.a-size-base').first().text();
    const published = $(metaElements).find('.a-text-normal').last().text();

    if (asin && title && imageUrl && productPath && !title.startsWith('Sponsored Ad')) {
      results.push({ asin, title, imageUrl, author, published, productPath });
    }
  });

  return results;
}

loadPage('How To Take Smart Notes Summary').then(parseAmazonSearchResults).then(console.info);

Promise { <pending> }


[
  {
    asin: 'B088TW4SPZ',
    title: 'How To Take Smart Notes Summary: A Summary of the Book That Teaches One Simple Technique To Boost Writing, Learning and Th...',
    imageUrl: 'https://m.media-amazon.com/images/I/71-l+gSWjxL._AC_UY218_.jpg',
    author: 'Upgraded Brain',
    published: 'May 18, 2020',
    productPath: '/How-Take-Smart-Notes-Summary-ebook/dp/B088TW4SPZ'
  },
  {
    asin: 'B09S17F4L8',
    title: 'Summary & Notes of How to Take Smart Notes : by Sönke Ahrens',
    imageUrl: 'https://m.media-amazon.com/images/I/71MinOP7E+L._AC_UY218_.jpg',
    author: 'David Alpha',
    published: 'Feb 7, 2022',
    productPath: '/Summary-Notes-How-Take-Smart-ebook/dp/B09S17F4L8'
  },
  {
    asin: 'B09J2TRLRG',
    title: 'Summary of How to Take Smart Notes - By Sönke Ahrens: The best book to master the art of making notes and boosting writing',
    imageUrl: 'https://m.media-amazon.com/images/I/71zlfE0aIiL._AC_UY218_.jpg',
    author: 'BestWriters .Club',
    published: 'Oct 8, 